In [72]:
#office computer

# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [73]:
import numpy as np
import torch

# 假設你有三個檔案：train, val, test
X_train_01 = np.load("C:/Users/wenwe/Desktop/Lab_QC/Wendy/qcnn_features_X_train_01p.npy")
X_val_01   = np.load("C:/Users/wenwe/Desktop/Lab_QC/Wendy/qcnn_features_X_val_01p.npy")
X_test_01  = np.load("C:/Users/wenwe/Desktop/Lab_QC/Wendy/qcnn_features_X_test_01p.npy")

# 轉成 torch tensor，放到 GPU（如果你有 GPU 的話）
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cpu")

X_train_01 = torch.tensor(X_train_01, dtype=torch.float32).to(device)
X_val_01   = torch.tensor(X_val_01, dtype=torch.float32).to(device)
X_test_01  = torch.tensor(X_test_01, dtype=torch.float32).to(device)

# Flatten 成 2D（如果需要的話）
X_train_flat = X_train_01.view(X_train_01.size(0), -1)
X_val_flat   = X_val_01.view(X_val_01.size(0), -1)
X_test_flat  = X_test_01.view(X_test_01.size(0), -1)

In [74]:
import numpy as np
import torch

# 假設你有三個檔案：train, val, test
y_train_01 = np.load("C:/Users/wenwe/Desktop/Lab_QC/Wendy/qcnn_labels_y_train_01p.npy")
y_val_01   = np.load("C:/Users/wenwe/Desktop/Lab_QC/Wendy/qcnn_labels_y_val_01p.npy")
y_test_01  = np.load("C:/Users/wenwe/Desktop/Lab_QC/Wendy/qcnn_labels_y_test_01p.npy")

# 轉成 torch tensor，放到 GPU（如果你有 GPU 的話）
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cpu")

y_train_01 = torch.tensor(y_train_01, dtype=torch.float32).to(device)
y_val_01   = torch.tensor(y_val_01, dtype=torch.float32).to(device)
y_test_01  = torch.tensor(y_test_01, dtype=torch.float32).to(device)

In [75]:
def torch_pca(X, k):
    """
    X: torch tensor of shape (N, D)，已經 flatten 過
    k: 降維後維度
    回傳: 降維後資料 (N, k)
    """
    # 中心化
    X_mean = X.mean(dim=0, keepdim=True)
    X_centered = X - X_mean

    # SVD 分解
    U, S, Vh = torch.linalg.svd(X_centered, full_matrices=False)

    # 投影到前 k 個主成分
    return X_centered @ Vh[:k].T

# 降維
n_components = 16
X_train_pca = torch_pca(X_train_flat, n_components)
X_val_pca   = torch_pca(X_val_flat, n_components)
X_test_pca  = torch_pca(X_test_flat, n_components)

print(X_train_flat.shape)
print("降維後的資料形狀：", X_train_pca.shape)  # (N_train, 16)
print("降維後的資料形狀：", X_val_pca.shape)
print("降維後的資料形狀：", X_test_pca.shape)

torch.Size([1180, 4096])
降維後的資料形狀： torch.Size([1180, 16])
降維後的資料形狀： torch.Size([270, 16])
降維後的資料形狀： torch.Size([269, 16])


In [76]:
import torch
import pennylane as qml

n_qubits = 16
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def feature_map(x):
    """
    x: torch.Tensor of shape (n_qubits,) 或 (d,) <= n_qubits
    """
    qml.AngleEmbedding(x, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [77]:
#測試
# x = torch.randn(n_qubits, device='cuda')  # 隨機輸入放在 GPU
# out = feature_map(x)
# print(out)  # 會是 torch.tensor([...])

In [78]:
#feature_map_batch = torch.vmap(feature_map)

# phi1 = feature_map_batch(X1)
# phi2 = feature_map(X2)

#K = torch.matmul(phi1, phi2.T) ** 2

In [79]:
def compute_kernel_batch(X1: torch.Tensor, X2: torch.Tensor) -> torch.Tensor:
    """
    使用 batch feature_map + matmul 計算整體 Gram matrix
    """
    phi1 = torch.stack(torch.vmap(feature_map)(X1))
    phi2 = torch.stack(torch.vmap(feature_map)(X2))
    return torch.matmul(phi1.T, phi2).to(torch.float32) ** 2

In [80]:
K_train = compute_kernel_batch(X_train_pca, X_train_pca)
K_val   = compute_kernel_batch(X_val_pca, X_train_pca)
K_test  = compute_kernel_batch(X_test_pca, X_train_pca)

print(K_train.shape , K_val.shape, K_test.shape)

torch.Size([1180, 1180]) torch.Size([270, 1180]) torch.Size([269, 1180])


In [81]:
print(K_train.dtype, K_train.min(), K_train.max())

torch.float32 tensor(3.6569e-11) tensor(127.8237)


In [82]:
K_min, K_max = K_train.min(), K_train.max()

K_train = (K_train - K_min)/(K_max - K_min)
K_val = (K_val- K_min)/(K_max - K_min)
K_test = (K_test- K_min)/(K_max - K_min)

In [83]:
# def quantum_kernel_fidelity_torch(x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
#     """
#     x1, x2: 單一樣本（1D torch tensors），shape: (n_features,)
#     回傳: torch scalar（kernel 值）
#     """
#     phi1 = feature_map(x1)  # torch tensor
#     phi2 = feature_map(x2)
#     return torch.dot(phi1, phi2) ** 2

In [84]:
# x1 = X_train_pca[0]
# x2 = X_train_pca[1]
# k_val = quantum_kernel_fidelity_torch(x1, x2)
# print(k_val)

In [85]:
# def compute_quantum_kernel_matrix_torch(X1: torch.Tensor,
#                                         X2: torch.Tensor,
#                                         kernel_func=quantum_kernel_fidelity_torch) -> torch.Tensor:
#     """
#     X1: (N1, d), X2: (N2, d)
#     kernel_func: 一個接受兩個向量的 kernel 函數
#     回傳: (N1, N2) 的 kernel matrix（torch.Tensor）
#     """
#     N1, N2 = X1.size(0), X2.size(0)
#     K = torch.zeros((N1, N2), device=X1.device)  # 保留在 GPU 上（若有）

#     for i in range(N1):
#         for j in range(N2):
#             K[i, j] = kernel_func(X1[i], X2[j])

#     return K

In [86]:
# K_train = compute_quantum_kernel_matrix_torch(X_train_pca, X_train_pca)
# K_val   = compute_quantum_kernel_matrix_torch(X_val_pca, X_train_pca)
# K_test  = compute_quantum_kernel_matrix_torch(X_test_pca, X_train_pca)

# print(K_train.shape, K_val.shape, K_test.shape)

# QSVM

In [88]:
import torch.nn as nn
import torch.nn.functional as F

class QSVMClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        # self.fc = nn.Linear(input_dim, 1)
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
      return self.fc(x)

In [89]:
model = QSVMClassifier(input_dim=K_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()

# 將標籤也轉成 tensor
y_train_tensor = torch.tensor(y_train_01, dtype=torch.float32).to(device).view(-1, 1)
y_val_tensor   = torch.tensor(y_val_01, dtype=torch.float32).to(device).view(-1, 1)

# 開始訓練
n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    preds = model(K_train)  # 輸入 kernel matrix 的每一列
    loss = loss_fn(preds, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


C:\Users\wenwe\AppData\Local\Temp\ipykernel_33820\2877454210.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train_01, dtype=torch.float32).to(device).view(-1, 1)
C:\Users\wenwe\AppData\Local\Temp\ipykernel_33820\2877454210.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val_tensor   = torch.tensor(y_val_01, dtype=torch.float32).to(device).view(-1, 1)


Epoch 0, Loss: 0.6925
Epoch 20, Loss: 0.6808
Epoch 40, Loss: 0.6702
Epoch 60, Loss: 0.6564
Epoch 80, Loss: 0.6388


In [90]:
print(y_train_tensor.shape)

torch.Size([1180, 1])


In [91]:
print(K_train.min().item(), K_train.max().item())

0.0 1.0


In [92]:
y_val_tensor   = torch.tensor(y_test_01, dtype=torch.float32).to(device).view(-1, 1)

C:\Users\wenwe\AppData\Local\Temp\ipykernel_33820\4178857608.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val_tensor   = torch.tensor(y_test_01, dtype=torch.float32).to(device).view(-1, 1)


In [93]:
from sklearn.metrics import accuracy_score

model.eval()
with torch.no_grad():
    val_logits = model(K_val)
    test_logits = model(K_test)

    val_probs = torch.sigmoid(val_logits)
    test_probs = torch.sigmoid(test_logits)

    val_preds = (val_probs > 0.5).int().cpu().numpy()
    test_preds = (test_probs > 0.5).int().cpu().numpy()

    y_val_true = y_val_01.cpu().numpy()
    y_test_true = y_test_01.cpu().numpy()
    
    val_acc = accuracy_score(y_val_true, val_preds)
    test_acc = accuracy_score(y_test_true, test_preds)

print(f"Validation Accuracy: {val_acc: .4f}" )
print(f"Test Accuracy: {test_acc : .4f}")

Validation Accuracy:  0.5148
Test Accuracy:  0.5093
